In [422]:
import pystac_client
import planetary_computer
import geopandas
import rich.table
import stackstac
import rasterio
import rioxarray
import sys
import os
import pyproj
import numpy
import dask.array as da
import datetime
from dateutil.parser import parse
import xarray

In [423]:
# Obiect de tip STAC
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
                                     modifier=planetary_computer.sign_inplace,)

In [424]:
time_range = "2023-01-01/2023-06-30"
bbox = [18.817924, 68.323238, 19.200386, 68.390350]
search = catalog.search(collections=["landsat-c2-l2"], 
                        bbox=bbox, 
                        datetime=time_range,
                        query={"eo:cloud_cover": {"lt": 25}}
                        )
items = search.get_all_items()
len(items)

items_final = []
for item in items:
    date_time = item.properties['datetime']
    month = int(date_time[5:7])
    if month in [5, 6, 7 , 8, 9]:
        items_final.append(item)

print(len(items_final))

features = [item.to_dict() for item in items_final]

print(features)




c:\Users\teona\AppData\Local\ESRI\conda\envs\learnenv\Lib\site-packages\pystac_client\item_search.py:903: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


8
[{'type': 'Feature', 'stac_version': '1.0.0', 'stac_extensions': ['https://stac-extensions.github.io/raster/v1.1.0/schema.json', 'https://stac-extensions.github.io/eo/v1.0.0/schema.json', 'https://stac-extensions.github.io/view/v1.0.0/schema.json', 'https://stac-extensions.github.io/projection/v1.1.0/schema.json', 'https://landsat.usgs.gov/stac/landsat-extension/v1.1.1/schema.json', 'https://stac-extensions.github.io/classification/v1.0.0/schema.json', 'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'], 'id': 'LC09_L2SP_198012_20230630_02_T1', 'geometry': {'type': 'Polygon', 'coordinates': [[[15.7582624, 69.4135028], [13.964908, 67.7937532], [18.0151184, 67.1275993], [20.057401, 68.7086745], [15.7582624, 69.4135028]]]}, 'bbox': [13.82449458, 67.07631554, 20.2512515, 69.41979446], 'properties': {'gsd': 30, 'created': '2023-07-11T09:15:38.772342Z', 'sci:doi': '10.5066/P9OGBGM6', 'datetime': '2023-06-30T10:28:32.231141Z', 'platform': 'landsat-9', 'proj:epsg': 32633, 'pro

In [425]:
df = geopandas.GeoDataFrame.from_features(items_final, crs="epsg:4326")
df

,geometry,gsd,created,sci:doi,datetime,platform,proj:epsg,proj:shape,description,instruments,...,landsat:wrs_row,landsat:scene_id,landsat:wrs_path,landsat:wrs_type,view:sun_azimuth,landsat:correction,view:sun_elevation,landsat:cloud_cover_land,landsat:collection_number,landsat:collection_category
0,"POLYGON ((15.75826 69.4135, 13.96491 67.79375,...",30,2023-07-11T09:15:38.772342Z,10.5066/P9OGBGM6,2023-06-30T10:28:32.231141Z,landsat-9,32633,"[8461, 8421]",Landsat Collection 2 Level-2,"[oli, tirs]",...,012,LC91980122023181LGN00,198,2,171.150480,L2SP,44.695228,13.65,02,T1
1,"POLYGON ((17.76173 69.44908, 16.1552 67.92529,...",30,2023-07-17T09:21:06.288309Z,10.5066/P9C7I13B,2023-06-19T08:10:14.704288Z,landsat-7,32634,"[8281, 8891]",Landsat Collection 2 Level-2,[etm+],...,012,LE71960122023170SG100,196,2,132.246488,L2SP,39.325738,12.00,02,T1
2,"POLYGON ((15.40953 69.44777, 13.80457 67.92453...",30,2023-07-07T09:21:38.095757Z,10.5066/P9C7I13B,2023-06-07T08:21:01.130457Z,landsat-7,32633,"[8041, 8711]",Landsat Collection 2 Level-2,[etm+],...,012,LE71980122023158SG100,198,2,133.598689,L2SP,38.937527,1.00,02,T2
3,"POLYGON ((17.19084 69.41218, 15.40767 67.79105...",30,2023-06-02T09:11:30.734071Z,10.5066/P9OGBGM6,2023-05-22T10:22:24.895386Z,landsat-9,32634,"[8681, 8651]",Landsat Collection 2 Level-2,"[oli, tirs]",...,012,LC91970122023142LGN00,197,2,173.467738,L2SP,41.986636,8.40,02,T1
4,"POLYGON ((18.73367 69.41169, 16.95148 67.79052...",30,2023-05-19T09:12:25.806845Z,10.5066/P9OGBGM6,2023-05-15T10:16:20.012437Z,landsat-9,32634,"[8611, 8571]",Landsat Collection 2 Level-2,"[oli, tirs]",...,012,LC91960122023135LGN00,196,2,173.680602,L2SP,40.470725,12.34,02,T1
5,"POLYGON ((17.20304 69.41245, 15.41977 67.79119...",30,2023-05-10T09:19:47.030962Z,10.5066/P9OGBGM6,2023-05-06T10:22:33.180723Z,landsat-9,32634,"[8681, 8661]",Landsat Collection 2 Level-2,"[oli, tirs]",...,012,LC91970122023126LGN00,197,2,173.748791,L2SP,38.138946,0.58,02,T1
6,"POLYGON ((17.58086 69.45343, 15.95911 67.9299,...",30,2023-07-07T09:21:33.817329Z,10.5066/P9C7I13B,2023-05-06T08:16:05.334752Z,landsat-7,32634,"[8311, 8901]",Landsat Collection 2 Level-2,[etm+],...,012,LE71960122023126SG100,196,2,137.366513,L2SP,33.317488,2.00,02,T1
7,"POLYGON ((18.71446 69.44859, 17.10655 67.92447...",30,2023-07-07T09:21:32.409432Z,10.5066/P9C7I13B,2023-05-01T08:12:07.141496Z,landsat-7,32634,"[8231, 8851]",Landsat Collection 2 Level-2,[etm+],...,012,LE71960122023121SG100,196,2,137.839523,L2SP,31.911195,13.00,02,T1


In [426]:
# df.plot()
# df.explore()

In [427]:
bands_tif = ["atran.tif", "blue.tif", "cdist.tif", "coastal.tif", "drad.tif", "emis.tif", "emsd.tif", "green.tif", "lwir11.tif", "nir08.tif",
          "qa.tif", "qa_aerosol.tif", "qa_pixel.tif", "qa_radsat.tif", "red.tif", "swir16.tif", "swir22.tif", "trad.tif", "urad.tif"]
bands = []
for b in bands_tif:
    name_file, ext_file = os.path.splitext(b)
    bands.append(name_file)

# bands2 = [os.path.splitext(file)[0] for file in bands_tif]
# bands2

In [428]:
transformer = pyproj.Transformer.from_crs("EPSG:4326","EPSG:32634", always_xy=True)
bbox_32634 = transformer.transform_bounds(bbox[0], bbox[1], bbox[2], bbox[3])
print(bbox, bbox_32634)

[18.817924, 68.323238, 19.200386, 68.39035] (410068.57839700056, 7579985.7610239675, 426045.53110328136, 7587973.56726169)


In [429]:
items_final[0]

<Item id=LC09_L2SP_198012_20230630_02_T1>

In [430]:
# scene_ids = df['landsat:scene_id'].values

data_temp = None
for item in items_final:
    print(item.id)
    slc_off_date = datetime.datetime(year=2003, month=5, day=31).astimezone(tz=datetime.UTC)
    aqdate = parse(item.properties["datetime"])
    platform = item.properties["platform"]
    print("Data imagine ", aqdate, platform)

    if(platform == "landsat-7" and aqdate > slc_off_date  ):
        continue

    scene_id = item.id
    folder = os.path.join(r"D:\Disertatie\Out", scene_id)
    os.makedirs(folder, exist_ok=True)
    print(f"{scene_id}")
    

# for item in items_final:
    # Pentru fiecare ID se creeaza un folder; se verifica daca folder exista inainte sa fie creat
    # pentru fiecare banda se salveaza un fisier cu numele benzii; salvarea se face in folderul aferent benzii respective
    


    data = (stackstac.stack([item], 
                            assets=bands,
                            epsg=32634,
                            resolution=30,
                            bounds_latlon=bbox,
                            # bounds_latlon=bbox, #taie imaginea fix pe bbox
                            chunksize=4096).where(lambda x: x> 0, other=numpy.nan))
    
    
    qa_pixel = data.sel(band="qa_pixel").values.astype(numpy.int16)
    # qa_radsat = data.sel(band='qa_radsat')
    cloud_bitmask = numpy.bitwise_and(qa_pixel, 1 << 5) != 0
    shadow_bitmask = numpy.bitwise_and(qa_pixel, 1 << 3) != 0
    snow_bitmask = numpy.bitwise_and(qa_pixel, 1 << 4) != 0

    all_masks = cloud_bitmask & shadow_bitmask & snow_bitmask

    
    data.compute()
    data_masked = data.where(~all_masks)

    for band in bands:

        try:
            print(f"Se descarca banda {band} din {scene_id}")
            ref_bnd_data = data.sel(band=band)
            
            bnd_data = numpy.squeeze(data.sel(band=band).values.astype(numpy.int16))
            # display(data.sel(band=band))
            # A SE APLICA CORECT MASCA PRIN CARE SE PASTREAZA PIXELII FARA NORI, UMBRE NORI si ZAPADA - all_masks contine valoro de TRUE unde sunt nori, umbre nori si zapada
            numpy.putmask(bnd_data, all_masks, 0)
            
            out_bnd = r"D:\Disertatie\Out\{}\{}.tif".format(scene_id, band)

            if os.path.exists(out_bnd):
                os.unlink(out_bnd)

            with rasterio.open(out_bnd, 
                               "w", 
                               driver="GTiff",  
                               height=bnd_data.shape[0],  
                               width=bnd_data.shape[1], 
                               count=1,  
                               transform=ref_bnd_data.transform, 
                               crs=ref_bnd_data.crs,
                               dtype=numpy.int16) as tiff:
                tiff.write(bnd_data, 1)
            # bnd_data.rio.to_raster(out_bnd)

        except Exception as e:
            print(f"Eroare la salvarea benzii {band} din {scene_id}", e)
            pass

LC09_L2SP_198012_20230630_02_T1
Data imagine  2023-06-30 10:28:32.231141+00:00 landsat-9
LC09_L2SP_198012_20230630_02_T1
Se descarca banda atran din LC09_L2SP_198012_20230630_02_T1
Se descarca banda blue din LC09_L2SP_198012_20230630_02_T1


C:\Users\teona\AppData\Local\Temp\ipykernel_6356\1829425252.py:53: RuntimeWarning: invalid value encountered in cast
  bnd_data = numpy.squeeze(data.sel(band=band).values.astype(numpy.int16))


Se descarca banda cdist din LC09_L2SP_198012_20230630_02_T1
Se descarca banda coastal din LC09_L2SP_198012_20230630_02_T1
Se descarca banda drad din LC09_L2SP_198012_20230630_02_T1
Se descarca banda emis din LC09_L2SP_198012_20230630_02_T1
Se descarca banda emsd din LC09_L2SP_198012_20230630_02_T1
Se descarca banda green din LC09_L2SP_198012_20230630_02_T1
Se descarca banda lwir11 din LC09_L2SP_198012_20230630_02_T1
Se descarca banda nir08 din LC09_L2SP_198012_20230630_02_T1
Se descarca banda qa din LC09_L2SP_198012_20230630_02_T1
Se descarca banda qa_aerosol din LC09_L2SP_198012_20230630_02_T1
Se descarca banda qa_pixel din LC09_L2SP_198012_20230630_02_T1
Se descarca banda qa_radsat din LC09_L2SP_198012_20230630_02_T1
Se descarca banda red din LC09_L2SP_198012_20230630_02_T1
Se descarca banda swir16 din LC09_L2SP_198012_20230630_02_T1
Se descarca banda swir22 din LC09_L2SP_198012_20230630_02_T1
Se descarca banda trad din LC09_L2SP_198012_20230630_02_T1
Se descarca banda urad din LC09_

C:\Users\teona\AppData\Local\Temp\ipykernel_6356\1829425252.py:53: RuntimeWarning: invalid value encountered in cast
  bnd_data = numpy.squeeze(data.sel(band=band).values.astype(numpy.int16))


Se descarca banda cdist din LC09_L2SP_197012_20230522_02_T1
Se descarca banda coastal din LC09_L2SP_197012_20230522_02_T1
Se descarca banda drad din LC09_L2SP_197012_20230522_02_T1
Se descarca banda emis din LC09_L2SP_197012_20230522_02_T1
Se descarca banda emsd din LC09_L2SP_197012_20230522_02_T1
Se descarca banda green din LC09_L2SP_197012_20230522_02_T1
Se descarca banda lwir11 din LC09_L2SP_197012_20230522_02_T1
Se descarca banda nir08 din LC09_L2SP_197012_20230522_02_T1
Se descarca banda qa din LC09_L2SP_197012_20230522_02_T1
Se descarca banda qa_aerosol din LC09_L2SP_197012_20230522_02_T1
Se descarca banda qa_pixel din LC09_L2SP_197012_20230522_02_T1
Se descarca banda qa_radsat din LC09_L2SP_197012_20230522_02_T1
Se descarca banda red din LC09_L2SP_197012_20230522_02_T1
Se descarca banda swir16 din LC09_L2SP_197012_20230522_02_T1
Se descarca banda swir22 din LC09_L2SP_197012_20230522_02_T1
Se descarca banda trad din LC09_L2SP_197012_20230522_02_T1
Se descarca banda urad din LC09_

C:\Users\teona\AppData\Local\Temp\ipykernel_6356\1829425252.py:53: RuntimeWarning: invalid value encountered in cast
  bnd_data = numpy.squeeze(data.sel(band=band).values.astype(numpy.int16))


Se descarca banda cdist din LC09_L2SP_196012_20230515_02_T1
Se descarca banda coastal din LC09_L2SP_196012_20230515_02_T1
Se descarca banda drad din LC09_L2SP_196012_20230515_02_T1
Se descarca banda emis din LC09_L2SP_196012_20230515_02_T1
Se descarca banda emsd din LC09_L2SP_196012_20230515_02_T1
Se descarca banda green din LC09_L2SP_196012_20230515_02_T1
Se descarca banda lwir11 din LC09_L2SP_196012_20230515_02_T1
Se descarca banda nir08 din LC09_L2SP_196012_20230515_02_T1
Se descarca banda qa din LC09_L2SP_196012_20230515_02_T1
Se descarca banda qa_aerosol din LC09_L2SP_196012_20230515_02_T1
Se descarca banda qa_pixel din LC09_L2SP_196012_20230515_02_T1
Se descarca banda qa_radsat din LC09_L2SP_196012_20230515_02_T1
Se descarca banda red din LC09_L2SP_196012_20230515_02_T1
Se descarca banda swir16 din LC09_L2SP_196012_20230515_02_T1
Se descarca banda swir22 din LC09_L2SP_196012_20230515_02_T1
Se descarca banda trad din LC09_L2SP_196012_20230515_02_T1
Se descarca banda urad din LC09_

C:\Users\teona\AppData\Local\Temp\ipykernel_6356\1829425252.py:53: RuntimeWarning: invalid value encountered in cast
  bnd_data = numpy.squeeze(data.sel(band=band).values.astype(numpy.int16))


Se descarca banda cdist din LC09_L2SP_197012_20230506_02_T1
Se descarca banda coastal din LC09_L2SP_197012_20230506_02_T1
Se descarca banda drad din LC09_L2SP_197012_20230506_02_T1
Se descarca banda emis din LC09_L2SP_197012_20230506_02_T1
Se descarca banda emsd din LC09_L2SP_197012_20230506_02_T1
Se descarca banda green din LC09_L2SP_197012_20230506_02_T1
Se descarca banda lwir11 din LC09_L2SP_197012_20230506_02_T1
Se descarca banda nir08 din LC09_L2SP_197012_20230506_02_T1
Se descarca banda qa din LC09_L2SP_197012_20230506_02_T1
Se descarca banda qa_aerosol din LC09_L2SP_197012_20230506_02_T1
Se descarca banda qa_pixel din LC09_L2SP_197012_20230506_02_T1
Se descarca banda qa_radsat din LC09_L2SP_197012_20230506_02_T1
Se descarca banda red din LC09_L2SP_197012_20230506_02_T1
Se descarca banda swir16 din LC09_L2SP_197012_20230506_02_T1
Se descarca banda swir22 din LC09_L2SP_197012_20230506_02_T1
Se descarca banda trad din LC09_L2SP_197012_20230506_02_T1
Se descarca banda urad din LC09_

In [431]:
input_folder = r"D:\Disertatie\Out"  
output_folder = r"D:\Disertatie\Composite"

band_order = {
    "costal.tif": 1,
    "blue.tif": 2,
    "green.tif": 3,
    "red.tif": 4,
    "nir08.tif": 5,
    "swir16.tif":6,
    "swir22.tif":7}
